In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
model = load_model('lstm_model_weather_24seq.h5')

In [3]:
df = pd.read_csv('johor_senai_weather_data.csv')
df

,date,temp_celsius,temp_fahrenheit,condition,wind_speed_mph,pressure,humidity(%)
0,2014-01-01 00:00:00,23.888889,75,Mostly Cloudy,1,29.74,94
1,2014-01-01 01:00:00,23.888889,75,Mostly Cloudy,2,29.74,94
2,2014-01-01 02:00:00,23.888889,75,Mostly Cloudy,1,29.71,94
3,2014-01-01 03:00:00,23.888889,75,Mostly Cloudy,1,29.71,94
4,2014-01-01 04:00:00,23.888889,75,Mostly Cloudy,1,29.68,94
...,...,...,...,...,...,...,...
95184,2024-09-24 20:00:00,25.000000,77,Fair,2,29.65,94
95185,2024-09-24 21:00:00,25.000000,77,Fair,3,29.68,94
95186,2024-09-24 22:00:00,25.000000,77,Fair,3,29.68,94
95187,2024-09-24 23:00:00,25.000000,77,Fair,1,29.71,94


In [4]:
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by=['date'],ignore_index=True, inplace=True)
df

,date,temp_celsius,temp_fahrenheit,condition,wind_speed_mph,pressure,humidity(%)
0,2014-01-01 00:00:00,23.888889,75,Mostly Cloudy,1,29.74,94
1,2014-01-01 01:00:00,23.888889,75,Mostly Cloudy,2,29.74,94
2,2014-01-01 02:00:00,23.888889,75,Mostly Cloudy,1,29.71,94
3,2014-01-01 03:00:00,23.888889,75,Mostly Cloudy,1,29.71,94
4,2014-01-01 04:00:00,23.888889,75,Mostly Cloudy,1,29.68,94
...,...,...,...,...,...,...,...
95184,2024-09-24 20:00:00,25.000000,77,Fair,2,29.65,94
95185,2024-09-24 21:00:00,25.000000,77,Fair,3,29.68,94
95186,2024-09-24 22:00:00,25.000000,77,Fair,3,29.68,94
95187,2024-09-24 23:00:00,25.000000,77,Fair,1,29.71,94


In [5]:
columns_to_drop = ['condition']

In [6]:
df_new = df.drop(columns=columns_to_drop)
df_new

,date,temp_celsius,temp_fahrenheit,wind_speed_mph,pressure,humidity(%)
0,2014-01-01 00:00:00,23.888889,75,1,29.74,94
1,2014-01-01 01:00:00,23.888889,75,2,29.74,94
2,2014-01-01 02:00:00,23.888889,75,1,29.71,94
3,2014-01-01 03:00:00,23.888889,75,1,29.71,94
4,2014-01-01 04:00:00,23.888889,75,1,29.68,94
...,...,...,...,...,...,...
95184,2024-09-24 20:00:00,25.000000,77,2,29.65,94
95185,2024-09-24 21:00:00,25.000000,77,3,29.68,94
95186,2024-09-24 22:00:00,25.000000,77,3,29.68,94
95187,2024-09-24 23:00:00,25.000000,77,1,29.71,94


In [7]:
# Extract additional time features
df_new['Year'] = df_new['date'].dt.year
df_new['Month'] = df_new['date'].dt.month
df_new['Day'] = df_new['date'].dt.day
df_new['Hour'] = df_new['date'].dt.hour

In [8]:
columns_to_drop = ['date']
df_new = df_new.drop(columns=columns_to_drop)

In [9]:
df_new

,temp_celsius,temp_fahrenheit,wind_speed_mph,pressure,humidity(%),Year,Month,Day,Hour
0,23.888889,75,1,29.74,94,2014,1,1,0
1,23.888889,75,2,29.74,94,2014,1,1,1
2,23.888889,75,1,29.71,94,2014,1,1,2
3,23.888889,75,1,29.71,94,2014,1,1,3
4,23.888889,75,1,29.68,94,2014,1,1,4
...,...,...,...,...,...,...,...,...,...
95184,25.000000,77,2,29.65,94,2024,9,24,20
95185,25.000000,77,3,29.68,94,2024,9,24,21
95186,25.000000,77,3,29.68,94,2024,9,24,22
95187,25.000000,77,1,29.71,94,2024,9,24,23


In [10]:
# Define sequence length
seq_length = 24
sequences = []

# Create sequences from df_new
for i in range(len(df_new) - seq_length):
    # Use df_new instead of df to create sequences
    sequence = df_new.iloc[i:i+seq_length].values.astype(np.float32)
    sequences.append(sequence)

# Convert list of sequences to NumPy array
x = np.array(sequences)

# Check the shape of the input data
print(x.shape)  # Should be (num_samples, 24, num_features)

(95165, 24, 9)


In [11]:
# Make predictions
predictions = model.predict(x, verbose=1)

# Check the shape of predictions
print(f'Predictions shape: {predictions.shape}')  # Should be (95165, 1) if predicting one value (temperature)

2974/2974 [==============================] - 7s 2ms/step
Predictions shape: (95165, 1)


In [12]:
# Create a DataFrame for the predictions without denormalization
predictions_df = pd.DataFrame(predictions, columns=['Predicted Temperature'])

In [13]:
# Save the predictions to a CSV file
predictions_df.to_csv('predicted_temperatures_lstm1.csv', index=False)

print("Normalized predictions saved to 'predicted_temperatures_lstm1.csv'")

Normalized predictions saved to 'predicted_temperatures_lstm1.csv'
